In [ ]:
import pandas as pd
mdfe = pd.read_csv('data/melb_data_fe.csv', sep=',')
mdfe['Date'] = pd.to_datetime(mdfe['Date'])
mdfe['Date'].dt.quarter.value_counts().iloc[1]

In [ ]:
cols_to_exclude = ['Car', 'Date', 'Rooms', 'Bathroom', 'Bedroom']
mdc = mdfe.copy()
max_counts = 150
for col in mdc.columns:
    if mdc[col].nunique() < max_counts and col not in cols_to_exclude:
        mdc[col] = mdc[col].astype('category') 
        
mdc.info()

In [ ]:
mdc.sort_values(by=['Price', 'Distance']).loc[::10,['Price', 'Distance']]

In [ ]:
mask1 = mdc['AreaRatio'] < -0.8
mask2 = mdc['Type'] == 'townhouse'
mask3 = mdc['SellerG'] == 'McGrath'

mdc[mask1 & mask2 & mask3].sort_values(by=['Date', 'AreaRatio'], ascending=[True, False], ignore_index= True).loc[:, ['Date', 'AreaRatio']]

In [ ]:
copy = mdc.sort_values(by='AreaRatio', ascending=False, ignore_index=True)
copy.loc[1558, 'BuildingArea'] 

In [ ]:
rooms = mdc['Rooms'] > 2
town = mdc['Type'] == 'townhouse'
mdc[rooms & town].sort_values(by=['Rooms', 'MeanRoomsSquare'], ascending=[True, False], ignore_index=True).loc[18, 'Price']

In [ ]:
mdc.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

In [ ]:
mdc.groupby('MonthSale')['Price'].agg(['count', 'mean', 'max']).sort_values(by='count', ascending=False)

In [ ]:
mdc.groupby('Regionname')['SellerG'].agg(['nunique', set])

In [ ]:
mdc.groupby('Rooms')['Price'].mean().sort_values(ascending=False)

In [ ]:
mdc.groupby('Regionname')['Lattitude'].agg(['std', 'min']).sort_values(by='std', ascending=True)

In [ ]:
time = pd.to_datetime('2017-05-01')
time2 = pd.to_datetime('2017-09-01')
mask = (time <= mdc['Date']) & (time2 <= mdc['Date'])
mdc[mask].groupby('SellerG')['Price'].sum().sort_values(ascending=True)
#mdc[mdc['SellerG'] == 'LITTLE']['Price'].max()

In [ ]:
mdc.groupby('Rooms')[['Price', 'BuildingArea']].median()

In [ ]:
mdc.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

In [ ]:
mdc.pivot_table('Price', 'Rooms', 'Type', fill_value=0).round()

In [ ]:
mdc.pivot_table('Price', 'Regionname', 'Weekend', aggfunc='count')

In [ ]:
mdc.pivot_table('Landsize','Regionname', 'Type', aggfunc=['median', 'mean'], fill_value=0)

In [ ]:
mdc.pivot_table(values='Price', index=['Method', 'Type'], columns='Regionname', aggfunc='median',fill_value=0)

In [ ]:
pivot = mdc.pivot_table(values='Landsize', index='Regionname', columns='Type', aggfunc=['median', 'mean'], fill_value=0)
display(pivot['mean']['unit'])

In [ ]:
mask = pivot['mean']['house'] < pivot['median']['house']
fipi = pivot[mask]
fipi

In [ ]:
#mdc.pivot_table(values='BuildingArea', index='Rooms', columns='Type').median()
mdc.groupby(['Type', 'Rooms'])['BuildingArea'].median().unstack()

In [ ]:
mdc.groupby(['SellerG', 'Type'])['Price'].median().sort_values(ascending=False).unstack()

In [29]:
rat1 = pd.read_csv('Data/movs/ratings1.csv', sep=',')
rat2 = pd.read_csv('Data/movs/ratings2.csv', sep=',')
movie = pd.read_csv('Data/movs/movies.csv', sep=',')
dates = pd.read_csv('Data/movs/dates.csv', sep=',')
ratings = pd.concat([rat1, rat2], ignore_index=True)
ratings = ratings.drop_duplicates(ignore_index=True)
rat_dat =  pd.concat([ratings, dates], axis=1)
rat_dat


,userId,movieId,rating,date
0,1,1,4.0,2000-07-30 18:45:03
1,1,3,4.0,2000-07-30 18:20:47
2,1,6,4.0,2000-07-30 18:37:04
3,1,47,5.0,2000-07-30 19:03:35
4,1,50,5.0,2000-07-30 18:48:51
...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12


In [30]:
joined = rat_dat.join(
    movie.set_index('movieId'),
    on='movieId',
    how='left'
)
joined.head()

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [31]:
merged = rat_dat.merge(movie, 
                       on='movieId', 
                       how='left')
merged.head()

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [38]:
items_df = pd.DataFrame({
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394],
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})

purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132],
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})

merged_df = items_df.merge(purchase_df, on='item_id', how='inner')
income = (merged_df['price'] * merged_df['stock_count']).sum()
income

19729490

In [64]:
import re
def get_year_release(arg):
    candidates = re.findall(r'\(\d{4}\)', arg)
    if len(candidates) > 0:
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        return None


mov_rat = pd.read_csv('Data/movs/ratings_movies.csv', sep=',')
mov_rat['year_release'] = mov_rat['title'].apply(get_year_release)
year = mov_rat['year_release'] == 2018
group = mov_rat[year].groupby('genres')['rating'].agg(['mean', 'count'])
group[group['count'] > 10].sort_values(by='mean', ascending=False)

#mov_rat.groupby('userId')['genres'].agg('nunique', 'max').sort_values(ascending=False)

#mov_rat.groupby('userId')['rating'].agg('min', 'mean').sort_values(ascending=False)

,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12
